# Server's utility function

Each hospital has some numbers servers and each server has its own utility function that aims to maximise. This utility function is defined as:

$$
U_k^H = e (\text{\# served}) + (1-e) (\text{idle time})
$$
    - idle time = current time - busy time
    - e = importance of # served over idle time
    - k = server id
    - H = hospital

To make the two parameters comparable I used their normalised values where:

$$
    \# \text{served} = \frac{\text{\# served}}{\text{\# all individuals}}
$$
$$
    \text{idle time} = \frac{\text{current time - busy time}}{\text{current time}}
$$



## Other things that can be done:
$$
U_k^H = e_k (\text{\# served}) + (1-e_k) (\text{idle time})
$$

$$
    \# \text{served} = k \times \frac{\text{\# served}}{\text{\# all individuals}}
$$

- Think about the relative server speed. Work out for each server the relative speed. 
- Incorporate fatigue (Use total runtime in idle time formula, somehow)
- $e$ can be a function of time that can be different for each server.
- Some classes of servers that each have diferent e_functions

In [1]:
import random

import numpy as np
import sympy as sym

import ambulance_game as abg

In [2]:
def display_server_utilities(Q, utility_functions, e_parameter=0.5):
    for index, server in enumerate(Q.nodes[2].servers):
        print(
            f"Server {server.id_number}: {utility_functions[index](Q, server, e_parameter=e_parameter)}"
        )

## Simulation example
- Each server has the its own service rate (not state dependent)
- Server $i$ has a service rate of: $μ_i = i + 2$

In [3]:
rates = {}
for server in range(3):
    rates[server + 1] = {(u, v): server + 2 for u in range(2) for v in range(4)
}

In [4]:
def random_server(srv, ind):
    return random.random()

Q = abg.simulation.simulate_model(
    lambda_2=2, 
    lambda_1=7,
    mu=rates, 
    num_of_servers=3,
    threshold=2, 
    system_capacity=3,
    buffer_capacity=1,
    runtime=1000,
    seed_num=0,
    server_priority_function=random_server,
)

## Utility function 1

$$
U_k = e (\text{\# served}) + (1-e) (\text{idle time})
$$

In [25]:
def utility_function_1(Q, server, e_parameter=0.5):
    individuals = len(server.served_inds)
    idle_time = Q.current_time - server.busy_time
    return e_parameter * individuals + (1 - e_parameter) * idle_time

In [25]:
display_server_utilities(
    Q=Q, 
    e_parameter=0.5, 
    utility_functions=[utility_function_1 for _ in range(3)]
)

Server 1: 856.2787811624771
Server 2: 1138.6460411803682
Server 3: 1420.442564215299


## Utility function 2

$$
    U_k = e \frac{\text{\# served}}{\text{\# all individuals}} + (1 - e) \frac{\text{current time - busy time}}{\text{current time}}
$$

In [26]:
def utility_function_2(Q, server, e_parameter=0.5):
    individuals = len(server.served_inds) / len(Q.nodes[-1].all_individuals)
    idle_time = (Q.current_time - server.busy_time) / Q.current_time
    return e_parameter * individuals + (1 - e_parameter) * idle_time

In [27]:
display_server_utilities(
    Q=Q, 
    e_parameter=0.5, 
    utility_functions=[utility_function_2 for _ in range(3)]
)

Server 1: 0.258744837406165
Server 2: 0.3361370904013411
Server 3: 0.4092319157568416


## Utility function 3

$$
    U_k = e (\bar{\mu_k}) + (1 - e) (\text{idle time})
$$

- Should that be the mean of the service rates??

In [7]:
def utility_function_3(Q, server, e_parameter=0.5):
    server_rates = np.mean(list(rates[server.id_number].values()))
    idle_time = (Q.current_time - server.busy_time) / Q.current_time
    return e_parameter * server_rates + (1 - e_parameter) * idle_time

In [8]:
display_server_utilities(
    Q=Q, 
    e_parameter=0.5,
    utility_functions=[utility_function_3 for _ in range(3)]
)

Server 1: 1.1347757652412165
Server 2: 1.6696422450405164
Server 3: 2.1994381013238518


## Utility function 4

$$
    U_k = e (\text{busy time}) + (1 - e) (\text{idle time})
$$

# Utility function 7

$$
    U_k = e (\text{proportion of inds accepted}) + (1 - e) (\text{proportion of server idle time})
$$

In [84]:
def utility_function_7(Qs, server_id, e_parameter=0.5):
    server_all_simulations = [Q.nodes[2].servers[server_id - 1] for Q in Qs]
    all_lost_individuals = [
        len(Q.rejection_dict[1][0]) + len(Q.rejection_dict[2][0]) for Q in Qs
    ]
    all_accepted_individuals = [
        len(Q.nodes[-1].all_individuals) for Q in Qs
    ]
    
    mean_proportion_accepted = np.mean(
        [
            accepted_inds / (accepted_inds + lost_inds)
            for accepted_inds, lost_inds in zip(all_accepted_individuals, all_lost_individuals)
        ]
    )

    idle_proportion = np.mean(
        [
            (Qs[Q_id].current_time - srv.busy_time) / Qs[Q_id].current_time
            for Q_id, srv in enumerate(server_all_simulations)
        ]
    )
    return e_parameter * mean_proportion_accepted + (1 - e_parameter) * idle_proportion

In [85]:
def random_server(srv, ind):
    return random.random()

Q = abg.simulation.simulate_model(
    lambda_2=0, 
    lambda_1=7,
    mu=4, 
    num_of_servers=3,
    threshold=2, 
    system_capacity=10,
    buffer_capacity=1,
    runtime=1000,
    seed_num=0,
    server_priority_function=random_server,
)

In [87]:
utility_function_7(Qs=[Q], server_id=1, e_parameter=0.5)

0.7042416248509114

In [68]:
len(Q.rejection_dict[1][0]) + len(Q.rejection_dict[2][0])

17

In [69]:
lost_inds = len(Q.rejection_dict[1][0]) + len(Q.rejection_dict[2][0])
len(Q.get_all_individuals()) / (lost_inds + len(Q.get_all_individuals()))

0.9975814482856736